In [1]:
import json

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/app/evaluation_v1/results/mismatched_evaluation_results_gpt1.json", 'r') as f :
    data1 = json.load(f) 

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/app/evaluation_v1/results/mismatched_evaluation_results_gpt2.json", 'r') as f :
    data2 = json.load(f)

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/app/evaluation_v1/results/mismatched_evaluation_results_gpt_base.json", 'r') as f :
    data3 = json.load(f)

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/app/evaluation_v1/results/mismatched_evaluation_results_gpt_base_2.json", 'r') as f :
    data4 = json.load(f)

In [12]:
import pandas as pd
from collections import Counter

# === Invert better_model label for data2 ===
def invert_label(label):
    label = label.strip()
    if label.lower() == "model 1":
        return "Model 2"
    elif label.lower() == "model 2":
        return "Model 1"
    else:
        return "Tie"

# === Build mapping: query -> better_model ===
def get_better_model_map(data, invert=False):
    mapping = {}
    for item in data:
        query = item["query"]
        better_model = item.get("better_model", "Tie").strip()
        if invert:
            better_model = invert_label(better_model)
        mapping[query] = better_model
    return mapping

# In data1: Model 1 = GPT, Model 2 = Qwen
# In data2: Model 1 = Qwen, Model 2 = GPT → so we invert data2
bm1_map = get_better_model_map(data1, invert=False)
bm2_map = get_better_model_map(data2, invert=True)

# === Compare results for common queries ===
counter = Counter()
records = []

common_queries = set(bm1_map.keys()) & set(bm2_map.keys())

for query in common_queries:
    bm1 = bm1_map[query]  # From data1
    bm2 = bm2_map[query]  # From data2 (already inverted)
    
    counter[(bm1, bm2)] += 1
    records.append({
        "query": query,
        "Better Model (Data1)": bm1,
        "Better Model (Data2 - inverted)": bm2
    })

# === Display summary cleanly ===
print("\n=== 🔍 GPT vs Qwen-FT Comparison Across Two Evaluation Calls ===\n")
print("Note: Model labels are aligned. GPT = Model 1 in both, Qwen = Model 2\n")
print("Comparison format: [Data1 → Data2]")

summary_table = []

for (m1, m2), count in sorted(counter.items()):
    label = f"{m1} → {m2}"
    summary_table.append((label, count))

# Create clean DataFrame for display
df_summary = pd.DataFrame(summary_table, columns=["Comparison", "Query Count"])
print(df_summary.to_string(index=False))

# === Confusion Matrix ===
df_compare = pd.DataFrame(records)
conf_matrix = pd.crosstab(
    df_compare["Better Model (Data1)"],
    df_compare["Better Model (Data2 - inverted)"],
    rownames=["Data1 "],
    colnames=["Data2 "],
    dropna=False
)

print("\n=== 📊 Confusion Matrix ===")
print(conf_matrix)


=== 🔍 GPT vs Qwen-FT Comparison Across Two Evaluation Calls ===

Note: Model labels are aligned. GPT = Model 1 in both, Qwen = Model 2

Comparison format: [Data1 → Data2]
       Comparison  Query Count
Model 1 → Model 1         1335
Model 1 → Model 2          163
    Model 1 → Tie          170
Model 2 → Model 1          191
Model 2 → Model 2         1091
    Model 2 → Tie          304
    Tie → Model 1          325
    Tie → Model 2          131
        Tie → Tie          778

=== 📊 Confusion Matrix ===
Data2    Model 1  Model 2  Tie
Data1                         
Model 1     1335      163  170
Model 2      191     1091  304
Tie          325      131  778


In [13]:
import pandas as pd
from collections import Counter

# === Invert better_model label for data3 ===
def invert_label(label):
    label = label.strip()
    if label.lower() == "model 1":
        return "Model 2"
    elif label.lower() == "model 2":
        return "Model 1"
    else:
        return "Tie"

# === Build mapping: query -> better_model ===
def get_better_model_map(data, invert=False):
    mapping = {}
    for item in data:
        query = item["query"]
        better_model = item.get("better_model", "Tie").strip()
        if invert:
            better_model = invert_label(better_model)
        mapping[query] = better_model
    return mapping

# In data3: Model 1 = Qwen-Base, Model 2 = GPT → so we invert data3
# In data4: Model 1 = GPT, Model 2 = Qwen-Base → already aligned
bm3_map = get_better_model_map(data3, invert=True)   # invert to make GPT = Model 1
bm4_map = get_better_model_map(data4, invert=False)  # already aligned

# === Compare results for common queries ===
counter = Counter()
records = []

common_queries = set(bm3_map.keys()) & set(bm4_map.keys())

for query in common_queries:
    bm1 = bm3_map[query]  # From data3 (after inversion)
    bm2 = bm4_map[query]  # From data4 (no inversion needed)
    
    counter[(bm1, bm2)] += 1
    records.append({
        "query": query,
        "Better Model (Data3)": bm1,
        "Better Model (Data4)": bm2
    })

# === Display summary cleanly ===
print("\n=== 🔍 GPT vs Qwen-Base Comparison Across Two Evaluation Calls ===\n")
print("Note: Model labels are aligned. GPT = Model 1 in both, Qwen-Base = Model 2\n")
print("Comparison format: [Data3 → Data4]")

summary_table = []
for (m1, m2), count in sorted(counter.items()):
    label = f"{m1} → {m2}"
    summary_table.append((label, count))

# Display as DataFrame
df_summary = pd.DataFrame(summary_table, columns=["Comparison", "Query Count"])
print(df_summary.to_string(index=False))

# === Confusion Matrix ===
df_compare = pd.DataFrame(records)
conf_matrix = pd.crosstab(
    df_compare["Better Model (Data3)"],
    df_compare["Better Model (Data4)"],
    rownames=["Data3"],
    colnames=["Data4"],
    dropna=False
)

print("\n=== 📊 Confusion Matrix ===")
print(conf_matrix)



=== 🔍 GPT vs Qwen-Base Comparison Across Two Evaluation Calls ===

Note: Model labels are aligned. GPT = Model 1 in both, Qwen-Base = Model 2

Comparison format: [Data3 → Data4]
       Comparison  Query Count
Model 1 → Model 1         3970
Model 1 → Model 2          352
    Model 1 → Tie          375
Model 2 → Model 1          480
Model 2 → Model 2         2821
    Model 2 → Tie          290
    Tie → Model 1          704
    Tie → Model 2          675
        Tie → Tie         1254

=== 📊 Confusion Matrix ===
Data4    Model 1  Model 2   Tie
Data3                          
Model 1     3970      352   375
Model 2      480     2821   290
Tie          704      675  1254


In [ ]:
#  Format model 1 and model 2 output to be same

import json

# === Load input JSON ===
input_path = "/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/app/evaluation_v1/mismatched_outputs_llama.json"   # Replace with your actual input file
output_path = "converted_output_llama.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === Function to transform model_2 ===
def restructure_model_2(model_2_dict):
    if not isinstance(model_2_dict, dict):
        print(f"⚠️ Skipping model_2 — expected dict but got {type(model_2_dict)}")
        return model_2_dict  # Return as-is if it's not a dict

    new_model_2 = {}

    for key, values in model_2_dict.items():
        if not isinstance(values, list):
            continue  # Skip malformed entries
        for val in values:
            if not isinstance(val, str):
                continue  # Skip non-string values
            if ':' in val:
                parts = val.split(':', 1)
                new_key = parts[0].strip()
                expansion = parts[1].strip()
            else:
                new_key = key
                expansion = val.strip()

            # Add to new_model_2
            if new_key not in new_model_2:
                new_model_2[new_key] = []
            if expansion not in new_model_2[new_key]:
                new_model_2[new_key].append(expansion)

    return new_model_2

# === Process each entry ===
for entry in data:
    if isinstance(entry, dict) and "model_2" in entry:
        entry["model_2"] = restructure_model_2(entry["model_2"])

# === Save output ===
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ Transformed model_2 structure saved to: {output_path}")

In [2]:
import pandas as pd
from collections import Counter
import json

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/mismatched_evaluation_results_gpt_llama_nano.json", 'r') as f :
    data5 = json.load(f)

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/mismatched_evaluation_results_gpt_llama_nano_1st.json", 'r') as f :
    data6 = json.load(f)

# === Invert better_model label for data3 ===
def invert_label(label):
    label = label.strip()
    if label.lower() == "model 1":
        return "Model 2"
    elif label.lower() == "model 2":
        return "Model 1"
    else:
        return "Tie"

# === Build mapping: query -> better_model ===
def get_better_model_map(data, invert=False):
    mapping = {}
    for item in data:
        query = item["query"]
        better_model = item.get("better_model", "Tie").strip()
        if invert:
            better_model = invert_label(better_model)
        mapping[query] = better_model
    return mapping

# In data5: Model 1 = Qwen-Base, Model 2 = GPT → so we invert data3
# In data6: Model 1 = GPT, Model 2 = Qwen-Base → already aligned
bm3_map = get_better_model_map(data6, invert=True)   # invert to make GPT = Model 1
bm4_map = get_better_model_map(data5, invert=False)  # already aligned

# === Compare results for common queries ===
counter = Counter()
records = []

common_queries = set(bm3_map.keys()) & set(bm4_map.keys())

for query in common_queries:
    bm1 = bm3_map[query]  # From data3 (after inversion)
    bm2 = bm4_map[query]  # From data4 (no inversion needed)
    
    counter[(bm1, bm2)] += 1
    records.append({
        "query": query,
        "Better Model (Data5)": bm1,
        "Better Model (Data6)": bm2
    })

# === Display summary cleanly ===
print("\n=== 🔍 GPT vs Llama-Ft Comparison Across Two Evaluation Calls ===\n")
print("Note: Model labels are aligned. GPT = Model 1 in both, Llama-Ft = Model 2\n")
print("Comparison format: [Data3 → Data4]")

summary_table = []
for (m1, m2), count in sorted(counter.items()):
    label = f"{m1} → {m2}"
    summary_table.append((label, count))

# Display as DataFrame
df_summary = pd.DataFrame(summary_table, columns=["Comparison", "Query Count"])
print(df_summary.to_string(index=False))

# === Confusion Matrix ===
df_compare = pd.DataFrame(records)
conf_matrix = pd.crosstab(
    df_compare["Better Model (Data5)"],
    df_compare["Better Model (Data6)"],
    rownames=["Data5"], 
    colnames=["Data6"],
    dropna=False
)

print("\n=== 📊 Confusion Matrix ===")
print(conf_matrix)



=== 🔍 GPT vs Llama-Ft Comparison Across Two Evaluation Calls ===

Note: Model labels are aligned. GPT = Model 1 in both, Llama-Ft = Model 2

Comparison format: [Data3 → Data4]
       Comparison  Query Count
Model 1 → Model 1          284
Model 1 → Model 2         3175
    Model 1 → Tie          347
Model 2 → Model 1           23
Model 2 → Model 2         1192
    Model 2 → Tie          103
    Tie → Model 1           22
    Tie → Model 2          402
        Tie → Tie          736

=== 📊 Confusion Matrix ===
Data6    Model 1  Model 2  Tie
Data5                         
Model 1      284     3175  347
Model 2       23     1192  103
Tie           22      402  736


In [5]:
import pandas as pd
from collections import Counter
import json

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/mismatched_evaluation_results_gpt_llama_nano_1st_call.json", 'r') as f :
    data5 = json.load(f)

with open("/Users/rishabh.singh/Desktop/ai-search-retrieval-pipeline-poc-2/mismatched_evaluation_results_gpt_llama_nano_2nd_call.json", 'r') as f :
    data6 = json.load(f)

# === Invert better_model label for data3 ===
def invert_label(label):
    label = label.strip()
    if label.lower() == "model 1":
        return "Model 2"
    elif label.lower() == "model 2":
        return "Model 1"
    else:
        return "Tie"

# === Build mapping: query -> better_model ===
def get_better_model_map(data, invert=False):
    mapping = {}
    for item in data:
        query = item["query"]
        better_model = item.get("better_model", "Tie").strip()
        if invert:
            better_model = invert_label(better_model)
        mapping[query] = better_model
    return mapping

# In data5: Model 1 = GPT, Model 2 = Llama-Ft → so we invert data3
# In data6: Model 1 = Llama-Ft, Model 2 = GPT → already aligned
bm3_map = get_better_model_map(data5, invert=False)   # invert to make GPT = Model 1
bm4_map = get_better_model_map(data6, invert=True)  # already aligned

# === Compare results for common queries ===
counter = Counter()
records = []

common_queries = set(bm3_map.keys()) & set(bm4_map.keys())

for query in common_queries:
    bm1 = bm3_map[query]  # From data3 (after inversion)
    bm2 = bm4_map[query]  # From data4 (no inversion needed)
    
    counter[(bm1, bm2)] += 1
    records.append({
        "query": query,
        "Better Model (Data5)": bm1,
        "Better Model (Data6)": bm2
    })

# === Display summary cleanly ===
print("\n=== 🔍 GPT vs Llama-Ft Comparison Across Two Evaluation Calls ===\n")
print("Note: Model labels are aligned. GPT = Model 1 in both, Llama-Ft = Model 2\n")
print("Comparison format: [Data5 → Data6]")

summary_table = []
for (m1, m2), count in sorted(counter.items()):
    label = f"{m1} → {m2}"
    summary_table.append((label, count))

# Display as DataFrame
df_summary = pd.DataFrame(summary_table, columns=["Comparison", "Query Count"])
print(df_summary.to_string(index=False))

# === Confusion Matrix ===
df_compare = pd.DataFrame(records)
conf_matrix = pd.crosstab(
    df_compare["Better Model (Data5)"],
    df_compare["Better Model (Data6)"],
    rownames=["Data5"], 
    colnames=["Data6"],
    dropna=False
)

print("\n=== 📊 Confusion Matrix ===")
print(conf_matrix)



=== 🔍 GPT vs Llama-Ft Comparison Across Two Evaluation Calls ===

Note: Model labels are aligned. GPT = Model 1 in both, Llama-Ft = Model 2

Comparison format: [Data5 → Data6]
       Comparison  Query Count
Model 1 → Model 1          695
Model 1 → Model 2          199
    Model 1 → Tie          141
Model 2 → Model 1         1420
Model 2 → Model 2         1801
    Model 2 → Tie          454
    Tie → Model 1          229
    Tie → Model 2          166
        Tie → Tie          516

=== 📊 Confusion Matrix ===
Data6    Model 1  Model 2  Tie
Data5                         
Model 1      695      199  141
Model 2     1420     1801  454
Tie          229      166  516
